<a href="https://colab.research.google.com/github/darshit001/Auction-System/blob/main/4_LangGraph(supervisor_agent_type_of_graph).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from  langchain_groq import ChatGroq

from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

llm=ChatGroq(api_key="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",model="llama-3.3-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7d8dea2b22d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7d8dea2e22d0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
# !pip install langgraph_supervisor
# !pip install -U duckduckgo-search

# !pip install langchain langchain_community langgraph

In [16]:
from langchain_community.tools import DuckDuckGoSearchRun

def web_search(query):
  """Searches DuckDuckGo using LangChain's DuckDuckGoSearchRun tool."""
  search=DuckDuckGoSearchRun()
  return search.invoke(query)

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

In [18]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

math_agent=create_react_agent(
    model=llm,
    tools=[add,multiply],
    name="math_agent",
    prompt="you are my math expert. Always use one tool at a time."
)

In [19]:
research_agent=create_react_agent(
    model=llm,
    tools=[web_search],
    name="research_agent",
    prompt="You are world class research with access to web search. Do not do any math "
)

In [21]:
#supervisor will decide which tools called
work_flow=create_supervisor(
    [research_agent,math_agent],
    model=llm,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert."
        "for current event , use research_agent"
        "for math problems, use math_agent"
    )
)

In [23]:
app=work_flow.compile()

result= app.invoke({
    "messages":[
        {
            "role":"user",
            "content":"what is the capital of france?"
        }
    ]

})

In [24]:
result

{'messages': [HumanMessage(content='what is the capital of france?', additional_kwargs={}, response_metadata={}, id='fdf5daf6-337e-41ea-b661-9e231ddae1e5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qj9e', 'function': {'arguments': '{}', 'name': 'transfer_to_research_agent'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 285, 'total_tokens': 297, 'completion_time': 0.043636364, 'prompt_time': 0.019103476, 'queue_time': 0.249841812, 'total_time': 0.06273984}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_41c250edc7', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run-ef951d1c-cd8d-4447-bd21-17c0da9c0b2d-0', tool_calls=[{'name': 'transfer_to_research_agent', 'args': {}, 'id': 'call_qj9e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 285, 'output_tokens': 12, 'total_tokens': 297}),
  ToolMessage(content='Successfully transferred to research_agent', na

In [25]:
for m in result["messages"]:
  m.pretty_print()

================================ Human Message =================================

what is the capital of france?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_agent (call_qj9e)
 Call ID: call_qj9e
  Args:
================================= Tool Message =================================
Name: transfer_to_research_agent

Successfully transferred to research_agent
================================== Ai Message ==================================
Name: research_agent

The capital of France is Paris.
================================== Ai Message ==================================
Name: research_agent

Transferring back to supervisor
Tool Calls:
  transfer_back_to_supervisor (8d14eabe-94ed-4842-92b9-0a9c0b845f00)
 Call ID: 8d14eabe-94ed-4842-92b9-0a9c0b845f00
  Args:
================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred bac

In [27]:
app=work_flow.compile()

result= app.invoke({
    "messages":[
        {
            "role":"user",
            "content":"what is the weather of Ahmedabad Today, multiply it by 2 and add 5"
        }
    ]

})

for m in result["messages"]:
  m.pretty_print()

================================ Human Message =================================

what is the weather of Ahmedabad Today, multiply it by 2 and add 5
================================== Ai Message ==================================
Name: research_agent

I can't answer that question as it involves math operations which are not allowed. However, I can provide you with the current weather in Ahmedabad. According to the latest forecast, it is partly cloudy and hot in Ahmedabad today. If you want to know the exact temperature, I can provide you with that information.
================================== Ai Message ==================================
Name: research_agent

Transferring back to supervisor
Tool Calls:
  transfer_back_to_supervisor (0d336a49-6fe5-4446-81fe-2065dc31fda8)
 Call ID: 0d336a49-6fe5-4446-81fe-2065dc31fda8
  Args:
================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor
==